<a href="https://colab.research.google.com/github/seema-narvare/seema-narvare/blob/main/%2C_SIFT%2C_SURF_and_AKAZE_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import cv2
import numpy as np
from scipy.spatial.distance import pdist



from Ransac import Ransac
class MatchFeatures:

    descriptors = None
    key_points = None
    distances = None
    gPoint1 = None
    gPoint2 = None
    cRectangle = None

    def __init__(self, key_points, descriptors, distances):
        self.key_points = key_points
        self.descriptors = descriptors
        self.distances = distances
        self.Match()

    def Match(self):
        bf = cv2.BFMatcher(self.distances)
        matches = bf.knnMatch(self.descriptors, self.descriptors, k=10)
        ratio = 0.5
        mkp1, mkp2 = [], []

        for m in matches:
            j = 1

            while m[j].distance < ratio * m[j + 1].distance:
                j = j + 1

            for k in range(1, j):
                temp = m[k]

                if pdist(np.array([self.key_points[temp.queryIdx].pt,
                                   self.key_points[temp.trainIdx].pt])) > 10:
                    mkp1.append(self.key_points[temp.queryIdx])
                    mkp2.append(self.key_points[temp.trainIdx])

        # remove the false matches
        self.gPoint1, self.gPoint2, self.cRectangle = Ransac(mkp1, mkp2)

ModuleNotFoundError: No module named 'Ransac'

# New Section

# New Section

In [9]:
import cv2

from Detector.AbstractDetector import AbstractDetector


class AkazeDetector(AbstractDetector):
    # red
    image = None
    key_points = None
    descriptors = None
    color = (255, 0, 0)
    distance = cv2.NORM_HAMMING

    def __init__(self, image):
        self.image = image
        self.detectFeature()
        super().__init__(self.image)

    # detect keypoints and descriptors
    def detectFeature(self):
        sift = cv2.AKAZE_create()
        gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        self.key_points, self.descriptors = sift.detectAndCompute(gray, None)
        print("akaze",len(self.descriptors[1]))

ModuleNotFoundError: No module named 'Detector'

In [ ]:
from math import sqrt

import cv2
import numpy as np


def Ransac(match_kp1, match_kp2):
    inliers1 = []
    inliers2 = []
    count, rec = 0, 0

    p1 = np.float32([kp1.pt for kp1 in match_kp1])
    p2 = np.float32([kp2.pt for kp2 in match_kp2])

    homography, status = cv2.findHomography(p1, p2, cv2.RANSAC, 5.0)

    inliers_thresold = 2.5  # Eşitlik kontrolü ile değişkenleri belirlemek için mesafe eşiği
    # Birinci anahtar noktanın projeksiyonundan ikinci kilit noktaya olan mesafe eşikten azsa, homografi modeline uyar.
    # inliers için yeni bir eşleşme veriseti oluşturulur eşleşmeleri çizdirmek için gerekli

    good_matches = []
    for i, m in enumerate(match_kp1):

        col = np.ones((3, 1), dtype=np.float64)
        col[0:2, 0] = m.pt
        col = np.dot(homography, col)
        col /= col[2, 0]

        #HOMOGRAFİ İLE NOKTA ARASINDA UZAKLIK HESABI YAPILIR
        distance = sqrt(pow(col[0, 0] - match_kp2[i].pt[0], 2) +
                        pow(col[1, 0] - match_kp2[i].pt[1], 2))

        if distance < inliers_thresold:
            count = count + 1

    if count * 2.5 < len(match_kp1):
        inliers_thresold = 339
        rec = 3

    for i, m in enumerate(match_kp1):

            col = np.ones((3, 1), dtype=np.float64)
            col[0:2, 0] = m.pt
            col = np.dot(homography, col)
            col /= col[2, 0]

            distance = sqrt(pow(col[0, 0] - match_kp2[i].pt[0], 2) +
                        pow(col[1, 0] - match_kp2[i].pt[1], 2))

            if distance < inliers_thresold:
                good_matches.append(cv2.DMatch(len(inliers1), len(inliers2), 0))
                inliers1.append(match_kp1[i])
                inliers2.append(match_kp2[i])

    print('# eslesme:                            \t', len(match_kp1))
    print('# Inliers yani verilen homografiye uyan eşleşmeler:                            \t', len(inliers1))

    good_points1 = np.float32([kp1.pt for kp1 in inliers1])
    good_points2 = np.float32([kp2.pt for kp2 in inliers2])

    return good_points1, good_points2, rec

In [ ]:
from abc import ABCMeta, abstractmethod

from Detector.MatchFeature.Match import MatchFeatures
from DrawFunctions.Rectangle import DrawRectangle


class AbstractDetector(metaclass=ABCMeta):
    key_points = None
    descriptors = None
    color = None
    image = None
    distance = None
    MatchFeatures = None
    Draw = None

    def __init__(self, image):
        self.image = image
        self.MatchFeatures = MatchFeatures(self.key_points, self.descriptors, self.distance)  # match points
        self.Draw = DrawRectangle(self.image, self.MatchFeatures.gPoint1, self.MatchFeatures.gPoint2, self.color, self.MatchFeatures.cRectangle)  # draw matches
        #  self.Draw = DrawLine(self.image,  self.MatchFeatures.gPoint1,  self.MatchFeatures.gPoint2, self.color) # from DrawFunctions.Line import DrawLine -> import it
        self.image = self.Draw.image

    # detect keypoints and descriptors
    @abstractmethod
    def detectFeature(self):
        pass

In [ ]:
import cv2

from Detector.AbstractDetector import AbstractDetector


# copy-move forgery detection with sift
class SiftDetector(AbstractDetector):
    # blue
    image = None
    key_points = None
    descriptors = None
    color = (0, 0, 255)
    distance = cv2.NORM_L2

    def __init__(self, image):
        self.image = image
        self.detectFeature()
        super().__init__(self.image)

    # detect keypoints and descriptors
    def detectFeature(self):
        sift = cv2.SIFT_create()
        gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        self.key_points, self.descriptors = sift.detectAndCompute(gray, None)

In [ ]:
import cv2

from Detector.AbstractDetector import AbstractDetector


# copy-move forgery detection with surf
class SurfDetector(AbstractDetector):
    # Green
    image = None
    key_points = None
    descriptors = None
    color = (0, 255, 0)
    distance = cv2.NORM_L2

    def __init__(self, image):
        self.image = image
        self.detectFeature()
        super().__init__(self.image)

    # feature detect(descriptors and keypoints)
    def detectFeature(self):
        sift = cv2.SIFT_create()  # burayı surf create yap
        gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        self.key_points, self.descriptors = sift.detectAndCompute(gray, None)

In [ ]:
from abc import ABCMeta, abstractmethod

class AbstractShape(metaclass=ABCMeta):

    @abstractmethod
    def draw(self, image, key_points1, key_points2, color): pass

In [ ]:
import cv2

from DrawFunctions.AbstractShape import AbstractShape


class DrawLine(AbstractShape):
    image = None
    key_points1 = None
    key_points2 = None
    color = None

    def __init__(self, image, keypoints1, keypoints2, color):
        self.image = image
        self.key_points1 = keypoints1
        self.key_points2 = keypoints2
        self.color = color
        self.draw()

    def draw(self, **kwargs):
        forgery = self.image.copy()
        for keypoint1, keypoint2 in zip(self.key_points1, self.key_points2):
            if len(self.key_points1) > 1:
                cv2.line(forgery, (int(keypoint1[0]), int(keypoint1[1])), (int(keypoint2[0]), int(keypoint2[1])), self.color, 1)

        self.image = forgery

In [ ]:
import cv2
import numpy as np
from scipy.stats import stats

from DrawFunctions.AbstractShape import AbstractShape


class DrawRectangle(AbstractShape):

    image = None
    keypoints1 = None
    keypoints2 = None
    color = None
    cRectangle = None

    def __init__(self, image, keypoints1, keypoints2, color, count_rectangle):
        self.image = image
        self.keypoints1 = keypoints1
        self.keypoints2 = keypoints2
        self.color = color
        self.cRectangle = count_rectangle  # counts of rectangle
        self.draw()

    def draw(self, **kwargs):
        new_image = self.image.copy()

        if self.cRectangle == 0:
            k1x, k2x = np.max(self.keypoints1, axis=0), np.max(self.keypoints2, axis=0)
            k1n, k2n = np.min(self.keypoints1, axis=0), np.min(self.keypoints2, axis=0)
            cv2.rectangle(new_image, (int(k2x[0]) + 10, int(k2n[1]) - 10), (int(k2n[0]) - 10, int(k2x[1]) + 10), self.color, 3)
            cv2.rectangle(new_image, (int(k1x[0]) + 10, int(k1n[1]) - 10), (int(k1n[0]) - 10, int(k1x[1]) + 10), self.color, 3)
            self.image = new_image
        # elif self.cRectangle == 3:
        #     point_list, z = np.zeros(len(self.keypoints1)), 0
        #     z2, z3, z4 = np.array([[0, 0]]), np.array([[0, 0]]), np.array([[0, 0]])
        #     for k1, k2 in zip(self.keypoints1, self.keypoints2):
        #         if len(self.keypoints1) > 1:
        #             p = (k1[0] - k2[0]) / (k1[1] - k2[1])
        #             point_list[z] = int(p)
        #             z = z + 1
        #     for k1, k2 in zip(self.keypoints1, self.keypoints2):
        #         if len(self.keypoints1) > 1:
        #             p = (k1[0] - k2[0]) / (k1[1] - k2[1])
        #             p = int(p)
        #             if p == max(point_list):
        #                 newrow = [k1[0], k1[1]]
        #                 z2 = np.vstack([z2, newrow])
        #             elif p < 0:
        #                 newrow = [k1[0], k1[1]]
        #                 z3 = np.vstack([z3, newrow])
        #                 newrow = [k2[0], k2[1]]
        #                 z4 = np.vstack([z4, newrow])
        #
        #     k1x, k11x, k2x = np.max(z3, axis=0), np.max(z2, axis=0), np.max(z4, axis=0)
        #     z2[0], z3[0], z4[0] = k11x, k1x, k2x
        #     k11n, k1n, k2n = np.min(z2, axis=0), np.min(z3, axis=0), np.min(z4, axis=0)
        #
        #     cv2.rectangle(new_image, (int(k2x[0]) + 10, int(k2n[1]) - 10), (int(k2n[0]) - 10, int(k2x[1]) + 10), self.color, 3)
        #     cv2.rectangle(new_image, (int(k11x[0]) + 10, int(k11n[1]) - 10), (int(k11n[0]) - 10, int(k11x[1]) + 10), self.color, 3)
        #     cv2.rectangle(new_image, (int(k1x[0]) + 10, int(k1n[1]) - 10), (int(k1n[0]) - 10, int(k1x[1]) + 10), self.color, 3)
        # self.image = new_image
        elif self.cRectangle == 3:
            egimlist, x = np.empty(0), 0
            reclist1, reclist2, reclist3 = np.empty(shape=[0, 2]), np.empty(shape=[0, 2]), np.empty(shape=[0, 2])
            for k1, k2 in zip(self.keypoints1, self.keypoints2):
                if len(self.keypoints1) > 1:
                    egim = (k1[0] - k2[0]) / (k1[1] - k2[1])
                    egim = int(egim)
                    egimlist = np.append(egimlist, [egim])
            mode = stats.mode(egimlist)

            while x != len(egimlist):
                if egimlist[x] == mode[0]:
                    egimlist = np.delete(egimlist, x)
                else:
                    x = x + 1
            mode2 = stats.mode(egimlist)
            for k1, k2 in zip(self.keypoints1, self.keypoints2):
                if len(self.keypoints1) > 1:
                    egim = (k1[0] - k2[0]) / (k1[1] - k2[1])
                    egim = int(egim)
                    if egim == mode[0]:
                        reclist1 = np.append(reclist1, [[k1[0], k1[1]]], axis=0)
                        reclist3 = np.append(reclist3, [[k2[0], k2[1]]], axis=0)
                    elif egim == mode2[0] or mode2[1] * 2 <= mode[1]:
                        reclist2 = np.append(reclist2, [[k1[0], k1[1]]], axis=0)

            k1x, k11x, k2x = np.max(reclist2, axis=0), np.max(reclist1, axis=0), np.max(reclist3, axis=0)
            k11n, k1n, k2n = np.min(reclist1, axis=0), np.min(reclist2, axis=0), np.min(reclist3, axis=0)

            cv2.rectangle(new_image, (int(k2x[0]) + 10, int(k2n[1]) - 10), (int(k2n[0]) - 10, int(k2x[1]) + 10),
                          self.color, 3)
            cv2.rectangle(new_image, (int(k11x[0]) + 10, int(k11n[1]) - 10), (int(k11n[0]) - 10, int(k11x[1]) + 10),
                          self.color, 3)
            cv2.rectangle(new_image, (int(k1x[0]) + 10, int(k1n[1]) - 10), (int(k1n[0]) - 10, int(k1x[1]) + 10),
                          self.color, 3)

        self.image = new_image

In [ ]:
import tkinter as tk  # tkinter dosya işlemleri için
from tkinter import *
from tkinter import filedialog

import cv2
import numpy as np
from PIL import Image
from PyQt5 import QtGui, QtWidgets, QtCore

from Detector.AkazeDetector import AkazeDetector
from Detector.SiftDetector import SiftDetector
from Detector.SurfDetector import SurfDetector
from GUI.Singleton import SingletonMeta


class Facade(metaclass=SingletonMeta):
    # boyut, resmi yeniden boyutlandırmak için stajyer numarasıdır, resim aynı en boy oranı korunarak yeniden boyutlandırılır
    # numpy Resim gösterimi ve yedeklenen resmi geri alma
    size = 720
    NPundo = np.empty((2, 2))
    NPimg = np.empty((2, 2))

    def setupUi(self, main_window):
        main_window.setObjectName("MainWindow")
        main_window.resize(720, 720)
        main_window.setCursor(QtGui.QCursor(QtCore.Qt.ArrowCursor))
        main_window.setMouseTracking(False)
        main_window.setStyleSheet("border-color: rgb(255, 255, 255);\n"
                                  "selection-background-color: rgb(135, 171, 255);\n"
                                  "background-color: rgb(255, 255, 255);\n")
        self.centralwidget = QtWidgets.QWidget(main_window)
        self.centralwidget.setObjectName("centralwidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.centralwidget)
        self.verticalLayout.setObjectName("verticalLayout_2")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setText("")
        self.label.setStyleSheet("QLabel{ background-color : rgb(204, 231, 232); color : black; }")
        self.label.setObjectName("label")
        self.verticalLayout.addWidget(self.label)
        main_window.setCentralWidget(self.centralwidget)
        self.menu_bar = QtWidgets.QMenuBar(main_window)
        self.menu_bar.setGeometry(QtCore.QRect(1, 5, 636, 21))
        self.menu_bar.setObjectName("menubar")
        self.q_menu = QtWidgets.QMenu(self.menu_bar)
        self.q_menu.setObjectName("menuMenu")
        main_window.setMenuBar(self.menu_bar)
        self.tool_bar = QtWidgets.QToolBar(main_window)
        self.tool_bar.setToolButtonStyle(QtCore.Qt.ToolButtonTextUnderIcon)
        self.tool_bar.setObjectName("toolBar")
        main_window.addToolBar(QtCore.Qt.LeftToolBarArea, self.tool_bar)
        self.action_Open = QtWidgets.QAction(main_window)
        self.action_Open.setObjectName("actionOpen")
        self.action_Undo = QtWidgets.QAction(main_window)
        icon = QtGui.QIcon()
        icon.addPixmap(QtGui.QPixmap("icons/refresh.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
        self.action_Undo.setIcon(icon)
        self.action_Undo.setObjectName("actionUndo")
        self.action_Save = QtWidgets.QAction(main_window)
        self.action_Save.setObjectName("actionSave")
        self.action_Surf = QtWidgets.QAction(main_window)
        icon1 = QtGui.QIcon()
        icon1.addPixmap(QtGui.QPixmap("icons/surf.jpeg"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
        self.action_Surf.setIcon(icon1)
        self.action_Surf.setObjectName("actionSurf")
        self.action_Akaze = QtWidgets.QAction(main_window)
        icon2 = QtGui.QIcon()
        icon2.addPixmap(QtGui.QPixmap("icons/akaze.jpeg"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
        self.action_Akaze.setIcon(icon2)
        self.action_Akaze.setObjectName("actionAkaze")
        self.action_Sift = QtWidgets.QAction(main_window)
        icon3 = QtGui.QIcon()
        icon3.addPixmap(QtGui.QPixmap("icons/sift.jpeg"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
        self.action_Sift.setIcon(icon3)
        self.action_Sift.setObjectName("actionSift")
        self.action_ZoomIn = QtWidgets.QAction(main_window)
        icon4 = QtGui.QIcon()
        icon4.addPixmap(QtGui.QPixmap("icons/zoom_in.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
        self.action_ZoomIn.setIcon(icon4)
        self.action_ZoomIn.setObjectName("actionZoomIn")
        self.action_Exit = QtWidgets.QAction(main_window)
        self.action_Exit.setObjectName("actionExit")
        self.action_ZoomOut = QtWidgets.QAction(main_window)
        icon5 = QtGui.QIcon()
        icon5.addPixmap(QtGui.QPixmap("icons/zoom_out.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
        self.action_ZoomOut.setIcon(icon5)
        self.action_ZoomOut.setObjectName("actionZoomOut")
        self.q_menu.addAction(self.action_Open)
        self.q_menu.addAction(self.action_Save)
        self.q_menu.addAction(self.action_Exit)
        self.menu_bar.addAction(self.q_menu.menuAction())
        self.tool_bar.addAction(self.action_Undo)
        self.tool_bar.addAction(self.action_Surf)
        self.tool_bar.addAction(self.action_Akaze)
        self.tool_bar.addAction(self.action_Sift)
        self.tool_bar.addAction(self.action_ZoomIn)
        self.tool_bar.addAction(self.action_ZoomOut)
        self.translateUi(main_window)
        QtCore.QMetaObject.connectSlotsByName(main_window)

    def translateUi(self, main_window):
        _translate = QtCore.QCoreApplication.translate
        main_window.setWindowTitle(_translate("MainWindow", "CMDF"))
        self.q_menu.setTitle(_translate("MainWindow", "Menu"))
        self.tool_bar.setWindowTitle(_translate("MainWindow", "İşlemler"))
        self.action_Open.setText(_translate("MainWindow", "Resim Seç"))
        self.action_Undo.setText(_translate("MainWindow", "Resmi Sıfırla"))
        self.action_Undo.setToolTip(_translate("MainWindow", "Yapılan İşlemleri Geri Al"))
        self.action_Save.setText(_translate("MainWindow", "Kaydet"))
        self.action_Surf.setText(_translate("MainWindow", "SURF"))
        self.action_Surf.setToolTip(_translate("MainWindow", "Surf Tabanlı Tespit."))
        self.action_Akaze.setText(_translate("MainWindow", "AKAZE"))
        self.action_Akaze.setToolTip(_translate("MainWindow", "Akaze Tabanlı Tespit."))
        self.action_Sift.setText(_translate("MainWindow", "SIFT"))
        self.action_Sift.setToolTip(_translate("MainWindow", "Sift Tabanlı Tespit."))
        self.action_ZoomIn.setText(_translate("MainWindow", "Yakınlaştır"))
        self.action_ZoomIn.setToolTip(_translate("MainWindow", "Resmi Yakınlaştır"))
        self.action_Exit.setText(_translate("MainWindow", "Çıkış"))
        self.action_Exit.setToolTip(_translate("MainWindow", "Programı Sonlandır."))
        self.action_ZoomOut.setText(_translate("MainWindow", "Uzaklaştır"))
        self.action_ZoomOut.setToolTip(_translate("MainWindow", "Resmi Uzaklaştır"))

        # buton bağlantıları

        self.action_Open.triggered.connect(self.openImage)
        self.action_Akaze.triggered.connect(self.akazeDetector)
        self.action_Sift.triggered.connect(self.siftDetector)
        self.action_Undo.triggered.connect(self.undo)
        self.action_Exit.triggered.connect(self.exit)  # programı sonlandır
        self.action_Surf.triggered.connect(self.surfDetector)
        self.action_ZoomOut.triggered.connect(self.zoomOut)
        self.action_ZoomIn.triggered.connect(self.zoomIn)
        self.action_Save.triggered.connect(self.saveImage)

    @staticmethod
    def exit():
        exit()

    # yapılan işlemi geri al
    def undo(self):
        self.image = self.origImage.copy()
        self.showImage(self.image)

    # önceki işlemden önce yedekleme
    def backup(self):
        self.NPundo = self.NPimg

    # resmi yakınlaştır
    def zoomIn(self):
        self.size = int(self.size * 1.2)
        self.showImage(self.image)

    # resmi uzaklaştır
    def zoomOut(self):
        self.size = int(self.size / 1.2)
        self.showImage(self.image)

    def openImage(self):
        self.backup()
        root = tk.Tk()
        root.withdraw()
        root.filename = filedialog.askopenfilename(initialdir="~",
                                                   title="Dosya Seç",
                                                   filetypes=(("jpeg files", "*.jpeg"), ("all files", "*.*")))
        if root.filename:
            # img = Image.open(root.filename)
            self.image = cv2.imread(root.filename, cv2.IMREAD_COLOR)
            self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
            self.origImage = self.image.copy()
            self.showImage(self.origImage)

    # resmi kaydederken rengi bozuluyor düzenlenecek
    def saveImage(self):
        root = tk.Tk()
        root.withdraw()
        root.filename = filedialog.asksaveasfilename(initialdir="~",
                                                     title="Resmi Kaydet",
                                                     filetypes=(("jpeg files", "*.jpeg"), ("all files", "*.*")))
        if root.filename:
            try:
                save_img = Image.fromarray(self.NPimg.astype('uint8'))
                save_img.save(root.filename)
            except ValueError:
                save_img = Image.fromarray(self.NPimg.astype('uint8'))
                save_img.save(root.filename + '.png')

    # Qt etiketindeki görüntüyü gösterir, boyut self.size'den türetilir (daha büyük ve daha küçük olarak değiştirilebilir)

    def showImage(self, img_show):
        image_profile = QtGui.QImage(img_show, img_show.shape[1], img_show.shape[0], img_show.shape[1] * 3,
                                     QtGui.QImage.Format_RGB888)  # QImage object
        image_profile = image_profile.scaled(self.size, self.size, aspectRatioMode=QtCore.Qt.KeepAspectRatio,
                                             transformMode=QtCore.Qt.SmoothTransformation)   # görüntüyü ölçeklendirmek ve En Boy Oranını korumak için
        self.label.setPixmap(QtGui.QPixmap.fromImage(image_profile))

    def siftDetector(self):
        sift = SiftDetector(self.image)
        self.image = sift.image
        self.showImage(self.image)

    def akazeDetector(self):
        akaze = AkazeDetector(self.image)
        self.image = akaze.image
        self.showImage(self.image)

    def surfDetector(self):
        surf = SurfDetector(self.image)
        self.image = surf.image
        self.showImage(self.image)

In [ ]:
class SingletonMeta(type):

    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

In [ ]:
from PyQt5 import QtWidgets
from GUI.FacadeGui import Facade

if __name__ == '__main__':
    import sys

    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Facade()
    u2 = Facade()

    #singleton test
    if id(ui) == id(u2):
        print("Singleton çalışır, her iki değişken de aynı örneği içerir")

    else:
        print("birden fazla örnek, singleton çalışmadı!")

    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

